In [1]:
import ee
import geemap
import geemap.colormaps as cm
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipygee import *
import plotly.express as px
import pandas as pd

In [3]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(76.8006, 30.6238, 14)
#Map.zoom_to_me(zoom=14, add_marker=True)
Map

Map(center=[30.6238, 76.8006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [4]:
style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

nd_options = ['Vegetation Index (NDVI)', 
              'Modified Water Index (MNDWI)',
              'Soil Index (NDSI)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Vegetation Index (NDVI)', description='Normalized Difference Index:', style=style)

year_widget = widgets.IntSlider(min=2017, max=2021, value=2021, description='Selected year:', width=400, style=style)

Startdate = ['2018-01-01','2019-01-01','2020-01-01','2021-01-01']
#Startdate = ['2018','2019','2020','2021']
startdate = widgets.Dropdown(options=Startdate, value='2021-01-01', description='Startdate', style=style)

Enddate = ['2018-12-31','2019-12-31','2020-12-31','2021-12-31']

enddate = widgets.Dropdown(options=Enddate, value='2021-12-31', description='End Date', style=style)

first_band = widgets.Dropdown(
    description='1st band:',
    options=['B2', 'B3','B4','B8', 'B11', 'B12'],
    value='B8',
    style=style
)

second_band = widgets.Dropdown(
    description='2nd band:',
    options=['B2', 'B3','B4','B8', 'B11', 'B12'],
    value='B4',
    style=style
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

aoi_widget = widgets.Checkbox(
    value=True,
    description='Use user-drawn AOI',
    style=style
)

def aoi_change(change):
    Map.layers = Map.layers[:4]   
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    output_widget.clear_output()
    
aoi_widget.observe(aoi_change, names='value')

nd_color = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style=style
)

def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'B8'
        second_band.value = 'B4' 
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'B3'
        second_band.value = 'B11'   
    elif nd_indices.value == 'Soil Index (NDSI)':
        first_band.value = 'B11'
        second_band.value = 'B8'        
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
        
nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([nd_threshold]),
    widgets.HBox([startdate,enddate]),
    widgets.HBox([nd_indices, first_band, second_band]),
    submit
])

full_widget

In [6]:
import os
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'my_map.html')

In [10]:
Map.to_html(outfile=html_file, title='My Map', width='100%', height='600px')